In [1]:
import numpy as np
import glob
import re
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
cow_file_path = "../TUDarmstadt/Annotations/sideviews-cows2"
car_file_path = "../TUDarmstadt/Annotations/sideviews-cars"
#motorbike_file_path = "../TUDarmstadt/Annotations/motorbike-testset"
files = glob.glob(cow_file_path + "/*")
files.extend(glob.glob(car_file_path + "/*"))
files = np.random.permutation(files)

In [8]:
n, w, h, c = len(files), 224, 224, 3
images = np.zeros((n,h,w,c)).astype(int)
label_dict = {"cowSide":0, "carSide":1, "motorbikeSide":2}
classes = np.zeros(n).astype(int)
bboxes = np.zeros((n, 4)).astype(int)

for i, file in enumerate(files):
    with open(file) as f:
        for line in f:
            if len(re.findall(r"Image filename", line)) != 0:
                path = line.split(":")[-1].rstrip("\n")
                path = "../" + re.sub(r"[\" ]", "", path)
                im = Image.open(path)
                im = im.resize((w,h), Image.BICUBIC)
                img = np.array(im)
                images[i] = img
            elif len(re.findall(r"Original label", line)) != 0:
                label = line.split(":")[-1].rstrip("\n")
                label = re.sub(r"[\" ]", "", label)
                classes[i] = label_dict[label]
            elif len(re.findall(r"Bounding box", line)) != 0:
                bbox = line.split(":")[-1]
                xmin, ymin, xmax, ymax = re.findall(r"(?<=[^\d])([\d]+)(?=[^\d])", bbox)
                xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
                bboxes[i] = np.array([xmin, ymin, xmax, ymax])